In [1]:
import numpy
import numpy as np
from scipy.optimize import linprog

In [2]:
def get_cell_type(row, col):
    if ((row == 0 or row == 4) and (col == 0 or col == 4)):
        return 0, 0.5; #corner
    elif(row == 0 or col == 0 or row == 4 or col == 4):
        return 1, 0.25; #edge
    else:
        return 2, 0; #middle
    
def get_neighbors(row, col):
    arr = []
    if (row+1<5):
        arr.append((row+1, col))
    if (col+1<5):
        arr.append((row, col+1))
    if (col-1>=0):
        arr.append((row, col-1))
    if (row-1>=0):
        arr.append((row-1, col))
    return arr

def get_state(row, col, action): #returning reward obtained as the 3rd argument
    if (row == 0 and col == 1):
        return (4, 1, 10)
    elif (row == 0 and col == 3):
        return (2, 3, 5)
    if (action == 0): #up
        if (row - 1 >= 0):
            return (row - 1, col, 0)
        
    if (action == 1): #right
        if (col + 1 < 5):
            return (row, col+1, 0)
        
    if (action == 2): #down
        if (row + 1 < 5):
            return (row + 1, col, 0)

    if (action == 3): #left
        if (col - 1 >= 0):
            return (row, col-1, 0)
    
    return (row, col, -1)
        

In [3]:
optimal_coeffs = numpy.zeros((100, 25))

In [4]:
row_num = 0;
gamma = 0.9;
b = [0]*100
for row in range(5):
    for col in range(5):
        for action in range(4):
            state_row, state_col, reward = get_state(row, col, action)
            arr_ind = 5*row + col
            state_ind = 5*state_row + state_col
            b[row_num] -= reward
            optimal_coeffs[row_num][arr_ind] -= 1
            optimal_coeffs[row_num][state_ind] += gamma
            row_num += 1

In [5]:
vstar = (linprog([1]*25, optimal_coeffs, b)).x

In [6]:
vstar = numpy.round(vstar, 1)
print(numpy.reshape(vstar, (5, 5)))

[[22.  24.4 22.  19.4 17.5]
 [19.8 22.  19.8 17.8 16. ]
 [17.8 19.8 17.8 16.  14.4]
 [16.  17.8 16.  14.4 13. ]
 [14.4 16.  14.4 13.  11.7]]


In [7]:
action_dict = {0:'U', 1:'R', 2:'D', 3:'L'}
optimal_actions = [["" for i in range (5)] for j in range(5)]
for row in range(5):
    for col in range(5):
        values = []
        for action in range(4):
            state_row, state_col, reward = get_state(row, col, action)
            values.append(vstar[state_row*5 + state_col])
        maxval = max(values)
        for action_num, val in enumerate(values):
            if (val == maxval):
                optimal_actions[row][col] = optimal_actions[row][col]+action_dict[action_num]

In [8]:
def get_action_matrix(po):
    action_dict = {0:'U', 1:'R', 2:'D', 3:'L'}
    action_matrix = ["" for i in range(len(po))]
    optimal_actions = [["" for i in range (5)] for j in range(5)]
    for i in range(1, len(po)-1):
        values = po[i]
        maxval = max(values)
        for j, val in enumerate(values):
            if (val == maxval):
                action_matrix[i] = action_matrix[i] + action_dict[j]
    mat = numpy.reshape(action_matrix, (4, 4))
    return mat

In [9]:
optimal_actions

[['R', 'URDL', 'L', 'URDL', 'L'],
 ['UR', 'U', 'UL', 'L', 'L'],
 ['UR', 'U', 'UL', 'UL', 'UL'],
 ['UR', 'U', 'UL', 'UL', 'UL'],
 ['UR', 'U', 'UL', 'UL', 'UL']]